In [1]:
import os
import random
import torch
import torchaudio
import librosa
import time
import numpy as np
from io import BytesIO
from tqdm import tqdm
from pydub import AudioSegment
from torch import nn, optim
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
from torchaudio.transforms import Resample
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import random_split
from torchaudio.transforms import Resample
from pydub import AudioSegment
from collections import Counter

In [2]:
import os
import random
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from pydub import AudioSegment
from io import BytesIO
import librosa
import numpy as np
from torchaudio.transforms import Resample

class AphasiaDataset(Dataset):
    def __init__(self, root_dir, target_sample_rate=16000, fft_size=512, 
                hop_length=256, win_length=512, min_duration=10, max_duration=30):
        self.root_dir = root_dir
        self.target_sample_rate = target_sample_rate
        self.fft_size = fft_size
        self.hop_length = hop_length
        self.win_length = win_length
        self.min_duration = min_duration * 1000  # convert to milliseconds
        self.max_duration = max_duration * 1000
        self.data = []

        # Collect raw data
        raw_data = []
        for label, folder in enumerate(["Aphasia", "Norm"]):
            folder_path = os.path.join(root_dir, folder)
            if not os.path.exists(folder_path):
                continue
            for file_name in os.listdir(folder_path):
                if file_name.endswith(".3gp"):
                    file_path = os.path.join(folder_path, file_name)
                    raw_data.append((file_path, label))

        # Audio processing and segmentation
        for file_path, label in raw_data:
            try:
                segments = self.process_audio(file_path)
                self.data.extend([(s, label) for s in segments])
            except Exception as e:
                print(f"Error processing {file_path}: {str(e)}")

        # Class balancing
        self.balance_classes()
        random.shuffle(self.data)

    def balance_classes(self):
        # Count the number of samples for each class
        class_counts = {}
        for _, label in self.data:
            class_counts[label] = class_counts.get(label, 0) + 1
        
        if len(class_counts) < 2:
            return
            
        # Determine weights for the sampler
        weights = [1/class_counts[label] for _, label in self.data]
        sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
        self.sampler = sampler

    def process_audio(self, file_path):
        audio = AudioSegment.from_file(file_path, format="3gp")
        duration = len(audio)  # in milliseconds
        segments = []

        # Process short files as a whole
        if duration < self.min_duration:
            return [self.create_spectrogram(audio)]
            
        # Split into segments
        start = 0
        while start + self.min_duration <= duration:
            segment_duration = min(
                random.randint(self.min_duration, self.max_duration),
                duration - start
            )
            end = start + segment_duration
            segment = audio[start:end]
            spectrogram = self.create_spectrogram(segment)
            if spectrogram is not None:
                segments.append(spectrogram)
            start = end  # The next segment starts at the end of the current one

        return segments

    def create_spectrogram(self, segment):
        try:
            # Convert to waveform
            buffer = BytesIO()
            segment.export(buffer, format="wav")
            buffer.seek(0)
            waveform, sample_rate = torchaudio.load(buffer)
            
            # Resampling
            if sample_rate != self.target_sample_rate:
                resampler = Resample(sample_rate, self.target_sample_rate)
                waveform = resampler(waveform)
            
            # Minimum length check
            if waveform.shape[1] < self.fft_size:
                return None
                
            # Create spectrogram
            y = waveform.numpy().squeeze()
            spectrogram = librosa.stft(
                y, 
                n_fft=self.fft_size,
                hop_length=self.hop_length,
                win_length=self.win_length
            )
            mag = np.abs(spectrogram).astype(np.float32)
            return torch.tensor(mag.T).unsqueeze(0)  # (1, T, F)
            
        except Exception as e:
            print(f"Spectrogram error: {str(e)}")
            return None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        spectrogram, label = self.data[idx]
        return spectrogram, torch.tensor(label, dtype=torch.long)

def pad_sequence(batch):
    # Handle empty batches
    if not batch:
        return torch.zeros(0), torch.zeros(0)
    
    spectrograms, labels = zip(*batch)
    
    # Determine maximum length
    max_len = max(s.shape[1] for s in spectrograms)
    freq_bins = spectrograms[0].shape[2]
    
    # Create padding tensor
    padded = torch.zeros(len(spectrograms), 1, max_len, freq_bins)
    for i, s in enumerate(spectrograms):
        padded[i, :, :s.shape[1], :] = s
        
    return padded, torch.stack(labels)

In [3]:
t = time.time()
dataset = AphasiaDataset("aphasia")
print(f"Time to create the dataset: {time.time() - t}")

Time to create the dataset: 198.43577003479004


was: Train samples: 648, Test samples: 162

Printed some numbers; on average roughly from 700 to 1800 now. This is basic chunking (i.e., without accounting for the fact that a chunk may contain silence).

In [4]:
# Remove balancing from the class constructor
del dataset.sampler  # if sampler was created in balance_classes()

# Split into train/test WITHOUT using sampler at this stage
total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create weights only for the training set
train_labels = [dataset.data[i][1] for i in train_dataset.indices]
class_counts = Counter(train_labels)

# Check that both classes are present
if len(class_counts) < 2:
    raise ValueError("One of the classes is missing in the training set")

class_weights = {label: 1.0 / count for label, count in class_counts.items()}
weights = [class_weights[label] for label in train_labels]

# Create sampler only for the training set
train_sampler = WeightedRandomSampler(weights, num_samples=len(train_dataset), replacement=True)

# Create DataLoaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    sampler=train_sampler,
    collate_fn=pad_sequence,
    drop_last=True  # add to ignore the last incomplete batch
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    collate_fn=pad_sequence,
    drop_last=True
)

In [5]:
for w, m in train_dataloader:
    print(w.shape, m.shape)
    print(m)
    break

torch.Size([4, 1, 1837, 257]) torch.Size([4])
tensor([0, 0, 0, 1])


In [6]:
print(len(train_dataloader), len(test_dataloader))

583 146


In [7]:
print(total_size)

2917


In [8]:
class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=True):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, input_seq):
        assert len(input_seq.size()) > 2
        reshaped_input = input_seq.contiguous().view(-1, input_seq.size(-1))
        output = self.module(reshaped_input)
        if self.batch_first:
            output = output.contiguous().view(input_seq.size(0), -1, output.size(-1))
        else:
            output = output.contiguous().view(-1, input_seq.size(1), output.size(-1))
        return output


class CNN_BLSTM(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN_BLSTM, self).__init__()
        # CNN
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, (3, 3), (1, 1), 1),
            nn.ReLU(),
            nn.Conv2d(16, 16, (3, 3), (1, 3), 1),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, (3, 3), (1, 1), 1),
            nn.ReLU(),
            nn.Conv2d(32, 32, (3, 3), (1, 3), 1),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3, 3), (1, 1), 1),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3, 3), (1, 3), 1),
            nn.ReLU(),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, (3, 3), (1, 1), 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, (3, 3), (1, 3), 1),
            nn.ReLU(),
        )

        # BLSTM
        self.blstm1 = nn.LSTM(512, 128, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.3)

        # Fully Connected
        self.flatten = TimeDistributed(nn.Flatten(), batch_first=True)
        self.dense1 = nn.Sequential(
            TimeDistributed(nn.Linear(in_features=256, out_features=128), batch_first=True),
            nn.ReLU(),
            nn.Dropout(0.3),
        )

        # Final classification layer
        self.final_linear = nn.Linear(128, num_classes)

    def forward(self, forward_input):
        conv1_output = self.conv1(forward_input)
        conv2_output = self.conv2(conv1_output)
        conv3_output = self.conv3(conv2_output)
        conv4_output = self.conv4(conv3_output)

        # Reshape for LSTM
        conv4_output = conv4_output.permute(0, 2, 1, 3)
        conv4_output = torch.reshape(conv4_output, (conv4_output.shape[0], conv4_output.shape[1], 4 * 128))

        # BLSTM
        blstm_output, _ = self.blstm1(conv4_output)
        blstm_output = self.dropout(blstm_output)

        # Fully Connected
        flatten_output = self.flatten(blstm_output)
        fc_output = self.dense1(flatten_output)
        # Apply final linear layer to the last time step
        logits = self.final_linear(fc_output[:, -1, :])  # [batch_size, num_classes]
  
        return logits  # [batch_size, num_classes]

In [9]:
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    acc = 0.0
    prec = 0.0
    rec = 0.0

    with torch.no_grad():
        for spectrograms, labels in tqdm(dataloader, desc="Validation"):
            spectrograms = spectrograms.to(device)
            labels = labels.to(device)
            
            outputs = model(spectrograms)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()  # get predicted classes
            labels = labels.cpu().numpy()
            acc += accuracy_score(labels, preds)
            prec += precision_score(labels, preds, zero_division=1)
            rec += recall_score(labels, preds, zero_division=1)

    acc = acc / len(dataloader)
    prec = prec / len(dataloader)
    rec = rec / len(dataloader)

    return acc, prec, rec

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 2
model = CNN_BLSTM(num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 50

In [11]:
for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss_list = []
    
    for spectrograms, labels in tqdm(train_dataloader, desc=f"Training Epoch {epoch}"):
        spectrograms = spectrograms.to(device)
        labels = labels.long().to(device)

        outputs = model(spectrograms)
        
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())

    avg_train_loss = torch.tensor(train_loss_list).mean().item()
    acc, prec, rec = evaluate_model(model, test_dataloader, criterion, device)
    
    print(f"Epoch {epoch}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")
    print(f"Validation Metrics - Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")

Validation: 100%|███████████████████████████████████████████████████████████| 146/146 [00:07<00:00, 19.98it/s]


Epoch 1/50 - Train Loss: 0.6936
Validation Metrics - Accuracy: 0.8699, Precision: 1.0000, Recall: 0.5616


Validation: 100%|███████████████████████████████████████████████████████████| 146/146 [00:07<00:00, 20.03it/s]


Epoch 2/50 - Train Loss: 0.6931
Validation Metrics - Accuracy: 0.8699, Precision: 1.0000, Recall: 0.5616


Validation: 100%|███████████████████████████████████████████████████████████| 146/146 [00:07<00:00, 19.23it/s]


...

Epoch 48/50 - Train Loss: 0.4044
Validation Metrics - Accuracy: 0.8134, Precision: 0.5833, Recall: 0.9087


Validation: 100%|███████████████████████████████████████████████████████████| 146/146 [00:09<00:00, 15.65it/s]


Epoch 49/50 - Train Loss: 0.3921
Validation Metrics - Accuracy: 0.8493, Precision: 0.6627, Recall: 0.9041


Validation: 100%|███████████████████████████████████████████████████████████| 146/146 [00:09<00:00, 15.56it/s]

Epoch 50/50 - Train Loss: 0.4690
Validation Metrics - Accuracy: 0.8271, Precision: 0.6495, Recall: 0.8733


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 2
model = CNN_BLSTM(num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 70

In [13]:
for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss_list = []
    
    for spectrograms, labels in tqdm(train_dataloader, desc=f"Training Epoch {epoch}"):
        spectrograms = spectrograms.to(device)
        labels = labels.long().to(device)

        outputs = model(spectrograms)
        
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())

    avg_train_loss = torch.tensor(train_loss_list).mean().item()
    acc, prec, rec = evaluate_model(model, test_dataloader, criterion, device)
    
    #scheduler.step(acc)
    
    print(f"Epoch {epoch}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")
    print(f"Validation Metrics - Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")

Validation: 100%|███████████████████████████████████████████████████████████| 146/146 [00:07<00:00, 19.14it/s]


Epoch 1/70 - Train Loss: 0.6935
Validation Metrics - Accuracy: 0.1301, Precision: 0.1301, Recall: 1.0000


Validation: 100%|███████████████████████████████████████████████████████████| 146/146 [00:08<00:00, 17.46it/s]


Epoch 2/70 - Train Loss: 0.6929
Validation Metrics - Accuracy: 0.8699, Precision: 1.0000, Recall: 0.5616


Validation: 100%|███████████████████████████████████████████████████████████| 146/146 [00:08<00:00, 16.88it/s]


...

Epoch 68/70 - Train Loss: 0.6780
Validation Metrics - Accuracy: 0.2894, Precision: 0.1376, Recall: 0.9304


Validation: 100%|███████████████████████████████████████████████████████████| 146/146 [00:10<00:00, 14.34it/s]


Epoch 69/70 - Train Loss: 0.6869
Validation Metrics - Accuracy: 0.3065, Precision: 0.1364, Recall: 0.9167


Validation: 100%|███████████████████████████████████████████████████████████| 146/146 [00:10<00:00, 14.35it/s]

Epoch 70/70 - Train Loss: 0.6805
Validation Metrics - Accuracy: 0.3082, Precision: 0.1427, Recall: 0.9395


Let's try use created split

In [14]:
import pandas as pd
df = pd.read_csv("splited_data/train_filenames.csv")
df

,file_name,label
0,N-0919-RAT-1-bike.wav,0
1,N-0919-RAT-1-robb.wav,0
2,N-0927-RAT-1-robb.wav,0
3,N-0927-RAT-1-bike.wav,0
4,N-0926-RAT-1-bike.wav,0
...,...,...
467,A-396-RAT-1-robb.wav,1
468,A-473-RAT-4-bike.wav,1
469,A-473-RAT-1-robb.wav,1
470,A-473-RAT-1-bike.wav,1


In [15]:
import os
import random
import torch
import pandas as pd
import torchaudio
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from pydub import AudioSegment
from io import BytesIO
import librosa
import numpy as np
from torchaudio.transforms import Resample
from collections import Counter

class AphasiaDataset(Dataset):
    def __init__(self, csv_file, root_dir, target_sample_rate=16000, fft_size=512, 
                 hop_length=256, win_length=512, min_duration=10, max_duration=30):
        self.root_dir = root_dir
        self.target_sample_rate = target_sample_rate
        self.fft_size = fft_size
        self.hop_length = hop_length
        self.win_length = win_length
        self.min_duration = min_duration * 1000  # in miliseconds
        self.max_duration = max_duration * 1000
        self.data = []

        # Load file list from CSV
        df = pd.read_csv(csv_file)

        # Audio processing and segmentation
        for _, row in df.iterrows():
            file_name, label = row['file_name'], row['label']
            file_path = self.find_audio_file(file_name, label)
            if file_path:
                try:
                    segments = self.process_audio(file_path)
                    self.data.extend([(s, label) for s in segments])
                except Exception as e:
                    print(f"Error processing {file_path}: {str(e)}")
        
        random.shuffle(self.data)
    
    def find_audio_file(self, file_name, label):
        """Find the file in the corresponding folder by label"""
        folder = "Aphasia" if label == 1 else "Norm"
        file_name = file_name[:-4]
        file_path = os.path.join(self.root_dir, folder, f"{file_name}.3gp")  
        if os.path.exists(file_path):
            return file_path
        print(f"Warning: {file_name}.3gp not found in {folder} folder.")
        return None

    def process_audio(self, file_path):
        audio = AudioSegment.from_file(file_path, format="3gp")
        duration = len(audio)  # in milliseconds
        segments = []

        if duration < self.min_duration:
            return [self.create_spectrogram(audio)]

        start = 0
        while start + self.min_duration <= duration:
            segment_duration = min(random.randint(self.min_duration, self.max_duration), duration - start)
            end = start + segment_duration
            segment = audio[start:end]
            spectrogram = self.create_spectrogram(segment)
            if spectrogram is not None:
                segments.append(spectrogram)
            start = end
        return segments

    def create_spectrogram(self, segment):
        try:
            buffer = BytesIO()
            segment.export(buffer, format="wav")
            buffer.seek(0)
            waveform, sample_rate = torchaudio.load(buffer)
            
            if sample_rate != self.target_sample_rate:
                resampler = Resample(sample_rate, self.target_sample_rate)
                waveform = resampler(waveform)
            
            if waveform.shape[1] < self.fft_size:
                return None
            
            y = waveform.numpy().squeeze()
            spectrogram = librosa.stft(y, n_fft=self.fft_size, hop_length=self.hop_length, win_length=self.win_length)
            mag = np.abs(spectrogram).astype(np.float32)
            return torch.tensor(mag.T).unsqueeze(0)  # (1, T, F)
        except Exception as e:
            print(f"Spectrogram error: {str(e)}")
            return None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        spectrogram, label = self.data[idx]
        return spectrogram, torch.tensor(label, dtype=torch.long)

def pad_sequence(batch):
    if not batch:
        return torch.zeros(0), torch.zeros(0)
    
    spectrograms, labels = zip(*batch)
    max_len = max(s.shape[1] for s in spectrograms)
    freq_bins = spectrograms[0].shape[2]
    
    padded = torch.zeros(len(spectrograms), 1, max_len, freq_bins)
    for i, s in enumerate(spectrograms):
        padded[i, :, :s.shape[1], :] = s
    
    return padded, torch.stack(labels)

In [16]:
root_dir = "aphasia"
train_dataset = AphasiaDataset("splited_data/train_filenames.csv", root_dir)
test_dataset = AphasiaDataset("splited_data/test_filenames.csv", root_dir)
val_dataset = AphasiaDataset("splited_data/val_filenames.csv", root_dir)

# Class balancing for train
train_labels = [label for _, label in train_dataset.data]
class_counts = Counter(train_labels)
if len(class_counts) < 2:
    raise ValueError("One of the classes is missing in the training set")

class_weights = {label: 1.0 / count for label, count in class_counts.items()}
weights = [class_weights[label] for _, label in train_dataset.data]
train_sampler = WeightedRandomSampler(weights, num_samples=len(train_dataset), replacement=True)

# DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=4, sampler=train_sampler, collate_fn=pad_sequence, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=pad_sequence, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=pad_sequence, drop_last=True)

In [17]:
print(len(train_dataloader), len(test_dataloader))

415 160


In [18]:
class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=True):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, input_seq):
        assert len(input_seq.size()) > 2
        reshaped_input = input_seq.contiguous().view(-1, input_seq.size(-1))
        output = self.module(reshaped_input)
        if self.batch_first:
            output = output.contiguous().view(input_seq.size(0), -1, output.size(-1))
        else:
            output = output.contiguous().view(-1, input_seq.size(1), output.size(-1))
        return output


class CNN_BLSTM(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN_BLSTM, self).__init__()
        # CNN
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 16, (3, 3), (1, 1), 1),
            nn.ReLU(),
            nn.Conv2d(16, 16, (3, 3), (1, 3), 1),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, (3, 3), (1, 1), 1),
            nn.ReLU(),
            nn.Conv2d(32, 32, (3, 3), (1, 3), 1),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3, 3), (1, 1), 1),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3, 3), (1, 3), 1),
            nn.ReLU(),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, (3, 3), (1, 1), 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, (3, 3), (1, 3), 1),
            nn.ReLU(),
        )

        # BLSTM
        self.blstm1 = nn.LSTM(512, 128, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.3)

        # Fully Connected
        self.flatten = TimeDistributed(nn.Flatten(), batch_first=True)
        self.dense1 = nn.Sequential(
            TimeDistributed(nn.Linear(in_features=256, out_features=128), batch_first=True),
            nn.ReLU(),
            nn.Dropout(0.3),
        )

        # Final classification layer
        self.final_linear = nn.Linear(128, num_classes)

    def forward(self, forward_input):
        conv1_output = self.conv1(forward_input)
        conv2_output = self.conv2(conv1_output)
        conv3_output = self.conv3(conv2_output)
        conv4_output = self.conv4(conv3_output)

        # Reshape for LSTM
        conv4_output = conv4_output.permute(0, 2, 1, 3)
        conv4_output = torch.reshape(conv4_output, (conv4_output.shape[0], conv4_output.shape[1], 4 * 128))

        # BLSTM
        blstm_output, _ = self.blstm1(conv4_output)
        blstm_output = self.dropout(blstm_output)

        # Fully Connected
        flatten_output = self.flatten(blstm_output)
        fc_output = self.dense1(flatten_output)
        # Apply final linear layer to the last time step
        logits = self.final_linear(fc_output[:, -1, :])  # [batch_size, num_classes]
  
        return logits  # [batch_size, num_classes]

In [19]:
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    acc = 0.0
    prec = 0.0
    rec = 0.0

    with torch.no_grad():
        for spectrograms, labels in tqdm(dataloader, desc="Validation"):
            spectrograms = spectrograms.to(device)
            labels = labels.to(device)
            
            outputs = model(spectrograms)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()  # get predicted classes
            labels = labels.cpu().numpy()
            acc += accuracy_score(labels, preds)
            prec += precision_score(labels, preds, zero_division=1)
            rec += recall_score(labels, preds, zero_division=1)

    acc = acc / len(dataloader)
    prec = prec / len(dataloader)
    rec = rec / len(dataloader)

    return acc, prec, rec

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 2
model = CNN_BLSTM(num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 50

In [21]:
for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss_list = []
    
    for spectrograms, labels in tqdm(train_dataloader, desc=f"Training Epoch {epoch}"):
        spectrograms = spectrograms.to(device)
        labels = labels.long().to(device)

        outputs = model(spectrograms)
        
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())

    avg_train_loss = torch.tensor(train_loss_list).mean().item()
    acc, prec, rec = evaluate_model(model, test_dataloader, criterion, device)
    
    print(f"Epoch {epoch}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")
    print(f"Validation Metrics - Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")

Validation: 100%|███████████████████████████████████████████████████████████| 160/160 [00:08<00:00, 19.77it/s]


Epoch 1/50 - Train Loss: 0.6931
Validation Metrics - Accuracy: 0.1484, Precision: 1.0000, Recall: 0.0000


Validation: 100%|███████████████████████████████████████████████████████████| 160/160 [00:08<00:00, 19.41it/s]


Epoch 2/50 - Train Loss: 0.6931
Validation Metrics - Accuracy: 0.8516, Precision: 0.8516, Recall: 1.0000


Validation: 100%|███████████████████████████████████████████████████████████| 160/160 [00:09<00:00, 17.56it/s]


...

Epoch 48/50 - Train Loss: 0.5801
Validation Metrics - Accuracy: 0.6766, Precision: 0.9646, Recall: 0.6401


Validation: 100%|███████████████████████████████████████████████████████████| 160/160 [00:11<00:00, 13.88it/s]


Epoch 49/50 - Train Loss: 0.6120
Validation Metrics - Accuracy: 0.5109, Precision: 0.9719, Recall: 0.4333


Validation: 100%|███████████████████████████████████████████████████████████| 160/160 [00:11<00:00, 14.25it/s]

Epoch 50/50 - Train Loss: 0.6232
Validation Metrics - Accuracy: 0.8781, Precision: 0.8969, Recall: 0.9604


In [22]:
acc, prec, rec = evaluate_model(model, val_dataloader, criterion, device)

Validation: 100%|███████████████████████████████████████████████████████████| 138/138 [00:10<00:00, 12.95it/s]


In [23]:
print(f"Validation Metrics - Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")

Validation Metrics - Accuracy: 0.8460, Precision: 0.9179, Recall: 0.9010
